In [1]:
from dataframe import Data
from io_utils import *
import matplotlib.pyplot as plt

In [2]:
from pandas import DataFrame

In [3]:
from prediction_evaluation import PredictionEvaluation

In [4]:
train_ob = load_ztf_train_data(head_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_master_HEAD.FITS', phot_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_master_PHOT.FITS')
test_ob = load_ztf_test_data(head_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_master_HEAD.FITS', phot_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_master_PHOT.FITS')

#train_ob = load_ztf_train_data()
#test_ob = load_ztf_test_data()

In [5]:
num_pc_components = 3

In [6]:
prediction_type_nos= [150,151]

In [7]:
color_band_dict = {'g':'C2', 'r':'C3'}

In [8]:
sample_numbers_train = {101:300,
               102:300,
               103:300, 
               112:300, 
               113:300, 
               114:300, 
               141:300,
               143:300,
               145:1000,
               150:1000, 
               151:1000, 
               160:300, 
               161:300, 
               162:300, 
               163:300, 
               164:300, 
               170:300, 
               180:300, 
               181:1000, 
               183:300, 
               190:300}
sample_numbers_train=None

In [9]:
sample_numbers_test = {101:150,
               102:150, 
               103:150, 
               112:150, 
               113:150, 
               114:150, 
               141:150, 
               143:150, 
               145:150, 
               150:500, 
               151:500, 
               160:150, 
               161:150, 
               162:150, 
               163:150, 
               164:150, 
               170:150, 
               180:150, 
               181:150, 
               183:150, 
               190:150}
sample_numbers_test=None

In [10]:
train_features_path = '/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_features_master_3_pcs_u_band.csv'
test_features_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_features_master_3_pcs_u_band_random_current_date_100_days_alert.csv'

In [11]:
train_features_path = None
test_features_path = None

In [12]:
use_random_current_date = True

In [13]:
#catch error if data_ob and path are not compatible 
train_ob.create_features_df(prediction_type_nos=prediction_type_nos, 
                            band_choice='u', 
                            num_pc_components=num_pc_components, 
                            min_flux_threshold =200,
                            #skip_random_date_event_types = [150,151],
                            #plot_predicted_curve_of_type=[150,151],
                            #plot_all_predictions=True,
                            color_band_dict=color_band_dict,
                            sample_numbers=sample_numbers_train, 
                            features_path=train_features_path,
                            #num_alert_days=50,
                            use_random_current_date=False)

/home/biswajit/anaconda3/envs/kilonova/lib/python3.7/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
/home/biswajit/anaconda3/envs/kilonova/lib/python3.7/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
/home/biswajit/anaconda3/envs/kilonova/lib/python3.7/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
/home/biswajit/anaconda3/envs/kilonova/lib/python3.7/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
/home/biswajit/anaconda3/envs/kilonova/lib/python3.7/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
/home/biswajit/anaconda3/envs/kilon

KeyboardInterrupt: 

In [ ]:
train_ob.discard_no_featues_events()

In [ ]:
test_ob.create_features_df(color_band_dict=color_band_dict, 
                           min_flux_threshold=200,
                           #save_fig_path = 'ZTF_partial_lt_curve/', 
                           prediction_type_nos=prediction_type_nos, 
                           band_choice='u',
                           #num_alert_days=50,
                           #plot_predicted_curve_of_type = [150,151],
                           #plot_all_predictions = True,
                           features_path = test_features_path,
                           num_pc_components=num_pc_components, 
                           sample_numbers=sample_numbers_test, 
                           use_random_current_date=False)

In [ ]:
print(np.sum(train_ob.features_df['y_true']==1))

In [ ]:
#train_ob.discard_no_featues_events()
test_ob.discard_no_featues_events()

In [ ]:
print(train_ob.features_df)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
use_number_of_points = True
bands = ['g', 'r']

cols = []
for i,band in enumerate(bands):
    for j in range(1,num_pc_components+1):
        if use_number_of_points:
            if j ==1:
                col_name = str(i)+'n'
                cols.append(col_name)
        col_name = str(i)+'pc'+str(j)
        cols.append(col_name)
        

        
x_train = train_ob.features_df[cols].values
y_train = train_ob.features_df['y_true'].values
print(cols)

clf=RandomForestClassifier(n_estimators=100, max_depth=11)
clf.fit(x_train,y_train)



In [ ]:
from sklearn.model_selection import GridSearchCV 

param_grid = {'n_estimators': [10,25,50,75,100],
               'max_depth': [3,4,5,6,7]}


rf_random = GridSearchCV(clf, param_grid, cv = 3)
# Fit the random search model
rf_random.fit(x_train, y_train)

In [ ]:
rf_random.

In [ ]:
x_test = test_ob.features_df[cols].values
y_test = test_ob.features_df['y_true'].values

y_pred_train=clf.predict(x_train)
y_pred_test=clf.predict(x_test)
y_score= clf.predict_proba(x_test)

In [ ]:
print(x_train.shape)

In [ ]:
print(np.sum(y_pred_train==1))

In [ ]:
print(np.sum(y_pred_test==1))

In [ ]:
print(len(y_pred_test))

train_save_df = train_ob.features_df.drop(['y_pred','y_true'],axis=1)
test_save_df = test_ob.features_df.drop(['y_pred','y_true'],axis=1)

train_save_df.to_csv('/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_features_master_3_pcs_u_band.csv', index=False)
test_save_df.to_csv('/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_features_master_3_pcs_u_band_random_current_date_30_days_alert.csv', index=False)

In [ ]:
#train_save_df.to_csv('/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_features_master_3_pcs_u_band.csv', index=False)
#test_save_df.to_csv('/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_features_master_3_pcs_u_band.csv', index=False)

In [ ]:
print(clf.predict([[0,0,0,0,0,0]]))

In [ ]:
pe = PredictionEvaluation(train_ob=train_ob, test_ob=test_ob,y_pred_train=y_pred_train, y_pred_test=y_pred_test)

In [ ]:
Final_Results/performance_statcase1_master_random_current_date_only_test_30_days_alert_neglect_non_estimated_curves.png'

In [ ]:
save_fig_base_path ="Final_Results/"
save_fig_postfix_path ='case1_master_random_current_date_train_and_test_50_days_alert_neglect_non_estimated_curves.png'

In [ ]:
fig = pe.plot_performance_statistics(y_score=y_score)
fig.savefig(save_fig_base_path+"performance_stat"+save_fig_postfix_path)
#fig.savefig("randomplt")
plt.show()
plt.close('all')
#pe.plot_performance_statistics(y_score=y_score, save_fig_path=save_fig_base_path+"performance_stat"+save_fig_postfix_path)

In [ ]:
x_limits = (-5000,5000)
y_limits = (-5000,5000)
fig = test_ob.plot_features_correlation(color_band_dict=color_band_dict, x_limits=x_limits, y_limits=y_limits)
fig.savefig(save_fig_base_path+"pc_correlation"+save_fig_postfix_path)
plt.show()

In [ ]:
#print(train_df)
x_limits = (-5000,5000)
y_limits = (-5000,5000)
fig = test_ob.plot_band_correlation(color_band_dict=color_band_dict, x_limits=x_limits, y_limits=y_limits)
fig.savefig(save_fig_base_path+"band_correlation"+save_fig_postfix_path)
plt.show()

In [ ]:
import pickle

In [ ]:
pkl_filename = "models/pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [ ]:
def compare_train_test(y_pred_train, y_train, y_pred, y_test, high_low=(0,1), bins=30):
    plt.hist(y_pred_train[y_train == 1],
                 color='r', alpha=0.5, range=high_low, bins=bins,
                 histtype='stepfilled', normed=True,
                 label='S (train)')
    plt.hist(y_pred_train[y_train == 0],
                 color='b', alpha=0.5, range=high_low, bins=bins,
                 histtype='stepfilled', normed=True,
                 label='B (train)')

    hist, bins = np.histogram(y_pred[y_test == 1],
                                  bins=bins, range=high_low, normed=True)
    scale = len(y_pred[y_test == 1]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    #width = (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.errorbar(center, hist, yerr=err, fmt='o', c='r', label='S (test)')

    hist, bins = np.histogram(y_pred[y_test == 0],
                                  bins=bins, range=high_low, normed=True)
    scale = len(y_pred[y_test == 0]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    #width = (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.errorbar(center, hist, yerr=err, fmt='o', c='b', label='B (test)')
    plt.xlabel("NN scores")
    plt.ylabel("Arbitrary units")
    plt.legend(loc='best')

In [ ]:
print(y_score.T[1])

In [ ]:
compare_train_test(y_pred_train, np.array(train_ob.features_df['y_true']), y_pred_test, test_ob.features_df['y_true'])